In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
import statsmodels as sm
import statsmodels.api as sma
import statsmodels.formula.api as smf
from scipy.stats import f

In [2]:
df_train, df_test = pd.read_csv("ks_train.csv", index_col=0), pd.read_csv("ks_test.csv", index_col=0)

In [3]:
def forward_selection_reg(df, label, sig_enter, sig_remove=None, stepwise=True):
    """
    Forward regression using F-statistics
    
    Parameters
    ----------
    df: pandas.DataFrame, contains columns ["y", "x1", ... , "x{p-1}"]
    sig_enter: float, p-value for f statistics threshold to enter
    sig_remove: float, p-value for f statistics threshold to drop (not required if stepwise=False)
    stepwise: bool, if use stepwise regression
    """
    # lambda sse computation
    sse_func = lambda x: np.power(x, 2).sum()
    # set container for selected predictors
    selected_feature = set()
    # number of predictors
    features = df.drop(columns=label).columns
    x_num = len(features)
    # number of samples
    n = len(df)
    
    # switch for break condition
    continue_select = True
    first_round = True
    prev_model = None
    cnt = 0
    while continue_select:
        cnt += 1
        print("-"*80)
        print("Iter", cnt)
        # turn off switch if no further progress
        continue_select = False
        # this iter;s candidated predictors
        candidates = [i for i in features if i not in selected_feature]
        # first iter
        if first_round:
            models_list = [smf.ols(formula=f"{label}~1+"+x, data=df).fit() for x in candidates]
            ssr_list = [model.mse_model for model in models_list]
        # non-first iter
        else:
            models_list = [smf.ols(formula=f"{label}~1+"+"+".join(selected_feature)+"+"+x, data=df).fit() for x in candidates]
            ssr_list = [sse_func(prev_model.resid) - sse_func(model.resid) for model in models_list]
        mse_list = [model.mse_resid for model in models_list]
        f_stat_list = [ssr / mse for mse, ssr in zip(mse_list, ssr_list)]
        
        # display result
        print("candidates:", candidates)
        # print("ssr:", ssr_list)
        # print("mse:", mse_list)
        # print("f_stat:", f_stat_list)
        print("p-value:", [1 - f.cdf(i, 1, n-len(selected_feature)-2) for i in f_stat_list])
        
        # get feature with max F-stat, select if exceed threshold
        max_index = np.argmax(f_stat_list)
        if 1 - f.cdf(f_stat_list[max_index], 1, n-len(selected_feature)-2) < sig_enter:
            continue_select = True
            selected_feature |= {candidates[max_index]}
            new_selected_model = models_list[max_index]
            print("selected predictor:", candidates[max_index])
        else:
            new_selected_model = prev_model
            print("No selected predictor.")
        
        if first_round:
            first_round = False
            prev_model = new_selected_model
            if stepwise:
                print("First round, skip dropping.")
        
        elif stepwise:
            # drop predictors
            temp_candidates = list(selected_feature)
            drop_model_list = [smf.ols(formula=f"{label}~1+"+"+".join(selected_feature-{x}), data=df).fit() for x in temp_candidates]
            drop_ssr_list = [sse_func(model.resid) - sse_func(new_selected_model.resid) for model in drop_model_list]
            drop_mse = new_selected_model.mse_resid
            drop_f_stat_list = [ssr / drop_mse for ssr in drop_ssr_list]

            # display result
            print("drop candidates:", temp_candidates)
            # print("drop ssr:", drop_ssr_list)
            # print("drop mse:", drop_mse)
            # print("drop f_stat:", drop_f_stat_list)
            print("p-value:", [1 - f.cdf(i, 1, n-len(selected_feature)-2) for i in drop_f_stat_list])

            min_index = np.argmin(drop_f_stat_list)
            if 1 - f.cdf(drop_f_stat_list[min_index], 1, n-len(temp_candidates)-1) > sig_remove:
                continue_select = True
                print("drop predictor:", temp_candidates[min_index])
                selected_feature -= {temp_candidates[min_index]}
                new_selected_model = drop_model_list[min_index]
            else:
                print("No dropped predictor.")

        # store as last iter's prev model
        prev_model = new_selected_model
    
    print("Finished.")
    print("-"*80)
    
    return selected_feature

In [4]:
selected_feature = forward_selection_reg(df_train, "price", 0.001, 0.002)

--------------------------------------------------------------------------------
Iter 1
candidates: ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'yr_sale', 'mon_sale', 'age']
p-value: [1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 6.416790536001304e-05, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 0.004436301463086267, 1.1102230246251565e-16, 1.1102230246251565e-16, 0.18241178810435377, 0.2586816782217686, 6.495914917081791e-13]
selected predictor: sqft_living
First round, skip dropping.
--------------------------------------------------------------------------------
Iter 2
candidates: ['bedrooms', 'bathrooms', 'sqft_lot', 'floors', 'waterfron

p-value: [1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16]
No dropped predictor.
--------------------------------------------------------------------------------
Iter 11
candidates: ['sqft_lot', 'floors', 'sqft_above', 'sqft_basement', 'renovated', 'long', 'sqft_living15', 'sqft_lot15', 'mon_sale']
p-value: [0.006193897054079134, 0.00010402429699218896, 1.08650310970404e-09, 1.08650310970404e-09, 7.300185332903553e-07, 3.255576919158898e-10, 1.2227838075418163e-07, 4.1440805120807056e-08, 0.022598627719578435]
selected predictor: long
drop candidates: ['yr_sale', 'bedrooms', 'lat', 'condition', 'long', 'view', 'waterfront', 'bathrooms', 'sqft_living', 'grade', 'age']
p-value: [1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 1.1102230246251565e-16, 3.2555813600509964e-10, 1.1

In [5]:
X_train, X_test = df_train[selected_feature], df_test[selected_feature]
y_train, y_test = df_train["price"], df_test["price"]

In [6]:
mean = X_train.mean()
std = X_train.std()
label_mean = y_train.mean()
label_std = y_test.std()
X_train, X_test, y_train, y_test = (X_train-mean)/std, (X_test-mean)/std, (y_train-label_mean)/label_std, (y_test-label_mean)/label_std

In [7]:
# lasso
lasso_m = Lasso(alpha=0.1)
lasso_m.fit(X_train, y_train)
mean_squared_error(y_test, lasso_m.predict(X_test))

0.36483916354156454

In [8]:
# ridge
ridge_m = Ridge(alpha=0.1)
ridge_m.fit(X_train, y_train)
mean_squared_error(y_test, ridge_m.predict(X_test))

0.305132734244172

In [9]:
lm = LinearRegression()
lm.fit(X_train, y_train)
mean_squared_error(y_test, lm.predict(X_test))

0.30513273420536846